# **Serving Function**

### ***Loading Libraries***

In [ ]:
 # Operating Systems
import os
import sys
import shutil
import datetime
import argparse

# Hyperparameter Tuning
import hypertune

# Tools
from setuptools import setup
from setuptools import find_packages

# Numerical Computing
import numpy as np

# Data Manipuation
import pandas as pd

# SciPy
import scipy
from scipy import stats

# Data Visualization
import itertools
import seaborn as sns
import matplotlib.pyplot as plt

# JavaScript Object Notation
import json

# BigQuery
from google.cloud import bigquery
from google.colab import auth

# Scikit-Learn
from sklearn.utils import shuffle

from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

# Extreme Gradient Boosting
import xgboost as xgb

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

# TensorFlow
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import Model
import tensorflow_datasets as tfds
from tensorflow_hub import KerasLayer
from tensorflow import feature_column as fc
from tensorflow.python.framework import dtypes
from tensorflow.keras.preprocessing import text
from tensorflow.keras.utils import to_categorical
from tensorflow_io.bigquery import BigQueryClient
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import callbacks, layers, models, utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Input, Flatten, Conv2D, MaxPooling2D

# Keras API
import kerastuner as kt

# Google Libraries
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials


In [ ]:
train_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train', 'test'),
    as_supervised=True)

In [ ]:
split = 3

dataset_train = train_data.window(split, split + 1).flat_map(lambda *ds: ds[0] if len(ds) == 1 else tf.data.Dataset.zip(ds))

dataset_validation = train_data.skip(split).window(1, split + 1).flat_map(lambda *ds: ds[0] if len(ds) == 1 else tf.data.Dataset.zip(ds))

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True, name='full_text')

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu', name='h1_dense'))
model.add(tf.keras.layers.Dense(1, name='positive_review_logits'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])



history = model.fit(dataset_train.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=dataset_validation.batch(512),
                    verbose=1)

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

### ***Predict Using The Trained Model In-Memory***

In [ ]:
review1 = 'The film is based on a prize-winning novel.' # neutral

review2 = 'The film is fast moving and has several great action scenes.' # positive

review3 = 'The film was very boring. I walked out half-way.' # negative

logits = model.predict(x=tf.constant([review1, review2, review3]))

print(logits)

In [ ]:
# From https://goshippo.com/blog/measure-real-size-any-python-object/

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    seen.add(obj_id)
    try:
        if isinstance(obj, dict):
            size += sum([get_size(v, seen) for v in obj.values()])
            size += sum([get_size(k, seen) for k in obj.keys()])
        elif hasattr(obj, '__dict__'):
            size += get_size(obj.__dict__, seen)
        elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
            size += sum([get_size(i, seen) for i in obj])
    except:
        pass
    return size
print('{} MB'.format(get_size(model)/(1000*1000)))

### ***Export The Model for Serving***

In [ ]:
shutil.rmtree('export/default', ignore_errors=True)

export_path = os.path.join('export', 'default', 'sentiment_{}'.format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))

model.save(export_path)

In [ ]:
!find export/default

In [ ]:
!ls -lh {export_path}/saved_model.pb

!ls -lh {export_path}/assets/tokens.txt

!ls -lh {export_path}/variables/variables.*

In [ ]:
!saved_model_cli show --dir {export_path} --tag_set serve --signature_def serving_default

In [ ]:
restored = tf.keras.models.load_model(export_path)

review1 = 'The film is based on a prize-winning novel.'
review2 = 'The film is fast moving and has several great action scenes.'
review3 = 'The film was very boring. I walked out half-way.'

infer = restored.signatures['serving_default']

outputs = infer(full_text_input=tf.constant([review1, review2, review3]))

logit = outputs['positive_review_logits']
print(logit)

In [ ]:
print(1 / (1 + np.exp(-logit)))

### ***Custom Serving Function***

In [ ]:
@tf.function(input_signature=[tf.TensorSpec([None], dtype=tf.string)])
def add_prob(reviews):
    logits = model(reviews, training=False)
    probs = tf.sigmoid(logits)
    return {
        'positive_review_logits' : logits,
        'positive_review_probability' : probs
    }
shutil.rmtree('export/probs', ignore_errors=True)
probs_export_path = os.path.join('export', 'probs', 'sentiment_{}'.format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))
model.save(probs_export_path, signatures={'serving_default': add_prob})

In [ ]:
!saved_model_cli show --dir {probs_export_path} --tag_set serve --signature_def serving_default

In [ ]:
restored = tf.keras.models.load_model(probs_export_path)

infer = restored.signatures['serving_default']

outputs = infer(reviews=tf.constant([review1, review2, review3]))

probs = outputs['positive_review_probability']
print(probs)

### ***Deploy to Cloud AI Platform Predictions***

In [ ]:
!find export/probs | head -2 | tail -1

In [ ]:
%%bash

MODEL_LOCATION=$(find export/probs | head -2 | tail -1)
MODEL_NAME=imdb
MODEL_VERSION=v1

TFVERSION=2.1
REGION=us-central1
BUCKET=ai-analytics-solutions-kfpdemo


modelname=$(gcloud ai-platform models list | grep -w "$MODEL_NAME")
echo $modelname
if [ -z "$modelname" ]; then
   echo "Creating model $MODEL_NAME"
   gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
else
   echo "Model $MODEL_NAME already exists"
fi

modelver=$(gcloud ai-platform versions list --model "$MODEL_NAME" | grep -w "$MODEL_VERSION")
echo $modelver
if [ "$modelver" ]; then
   echo "Deleting version $MODEL_VERSION"
   yes | gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
   sleep 10
fi


echo "Creating version $MODEL_VERSION from $MODEL_LOCATION"
gcloud ai-platform versions create ${MODEL_VERSION} \
       --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --staging-bucket gs://${BUCKET} \
       --runtime-version $TFVERSION

In [ ]:
%%writefile input.json
{"reviews": "The film is based on a prize-winning novel."}
{"reviews": "The film is fast moving and has several great action scenes."}
{"reviews": "The film was very boring. I walked out half-way."}

In [ ]:
!gcloud ai-platform predict --model imdb --json-instances input.json --version v1

In [ ]:
print(response['predictions'][0]['positive_review_probability'][0])

### ***Stateful Function***

In [ ]:
def stateless_fn(x):
    return 3*x + 15

class Stateless:
    def __init__(self):
        self.weight = 3
        self.bias = 15
    def __call__(self, x):
        return self.weight*x + self.bias

class State:
    def __init__(self):
        self.counter = 0
    def __call__(self, x):
        self.counter = self.counter + 1
        if self.counter % 2 == 0:
            return 3*x + 15
        else:
            return 3*x - 15

a1 = Stateless()
a = State()
print(stateless_fn(3))
print(stateless_fn(3))
print(a1(3))
print(a1(3))
print(a(3))
print(a(3))
print(a(3))
print(a(3))

### ***Online Prediction***

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL mlpatterns.neutral_3classes
OPTIONS(model_type='logistic_reg', input_label_cols=['health']) AS

SELECT
  IF(apgar_1min = 10, 'Healthy', IF(apgar_1min >= 8, 'Neutral', 'NeedsAttention')) AS health,
  plurality,
  mother_age,
  gestation_weeks,
  ever_born
FROM `bigquery-public-data.samples.natality`
WHERE apgar_1min <= 10

In [ ]:
%%bigquery
SELECT * FROM ML.PREDICT(MODEL mlpatterns.neutral_3classes,
    (SELECT
     2 AS plurality,
     32 AS mother_age,
     41 AS gestation_weeks,
     1 AS ever_born
    )
)

In [ ]:
%%bash
BUCKET=ai-analytics-solutions-kfpdemo
bq extract -m --destination_format=ML_TF_SAVED_MODEL mlpatterns.neutral_3classes  gs://${BUCKET}/export/baby_health

In [ ]:
%%bash

TFVERSION=1.15
REGION=us-central1
BUCKET=ai-analytics-solutions-kfpdemo
MODEL_LOCATION=gs://${BUCKET}/export/baby_health
MODEL_NAME=babyhealth
MODEL_VERSION=v1


modelname=$(gcloud ai-platform models list | grep -w "$MODEL_NAME")

echo $modelname
if [ -z "$modelname" ]; then
   echo "Creating model $MODEL_NAME"
   gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
else
   echo "Model $MODEL_NAME already exists"
fi


modelver=$(gcloud ai-platform versions list --model "$MODEL_NAME" | grep -w "$MODEL_VERSION")

echo $modelver
if [ "$modelver" ]; then
   echo "Deleting version $MODEL_VERSION"
   yes | gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
   sleep 10
fi


echo "Creating version $MODEL_VERSION from $MODEL_LOCATION"
gcloud ai-platform versions create ${MODEL_VERSION} \
       --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --staging-bucket gs://${BUCKET} \
       --runtime-version $TFVERSION

In [ ]:
%%writefile input.json
{"plurality": 2, "mother_age": 32, "gestation_weeks": 41, "ever_born": 1}

In [ ]:
!gcloud ai-platform predict --model babyhealth --json-instances input.json --version v1